In [1]:
from openbabel import openbabel

In [2]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [4]:
#matches_data = pd.read_csv('./MatchedDataNoStereo/FormoseAmm/NoStereoFormoseAmm.tsv', sep='\t')

In [5]:
#FormoseAmm_rels = pd.read_csv('./FormoseAmmData/FormoseAmm_rels.tsv', sep='\t')

In [6]:
def split(word):
    return[char for char in word]

In [7]:
def index_finder(Smiles, pathway, rels_data):
    index = 0
    rule = 0
    for i in range(len(rels_data['Reagent'])):
        if rels_data['Reagent'][i] == Smiles and rels_data['Formed/Produced'][i] == 1 and precursor_finder(rels_data['Index'][i], rels_data) not in pathway and precursor_finder(rels_data['Index'][i], rels_data) != rels_data['Reagent'][i]:
            index = rels_data['Index'][i] 
            rule = rels_data['Rule'][i]
            break
    return(index, rule)

In [8]:
def precursor_finder(index, rels_data):
    precursor = 0
    for i in range(len(rels_data['Index'])):
        if rels_data['Index'][i] == index and rels_data['Formed/Produced'][i] == -1:
            precursor = rels_data['Reagent'][i]
            break
    return(precursor)

In [9]:
def pathway_finder(Smiles, rels_data):
    pathway = [Smiles]
    #G0 = ['N', 'C=O', 'C(CO)=O', 'O']
    G0 = ['C=O', 'C(CO)=O']
    pathway_complete = False
    while pathway_complete == False:
        index = 0
        rule = 0
        index, rule = index_finder(pathway[-1], pathway, rels_data)
        if index == 0 or pathway[-1] in G0:
            pathway_complete == True
            break
        else:
            precursor = precursor_finder(index, rels_data)
            precursors = []
            products = []
            for i in range(len(rels_data['Index'])):
                if rels_data['Index'][i] == index and rels_data['Formed/Produced'][i] == -1:
                    precursors.append(rels_data['Reagent'][i])
                elif rels_data['Index'][i] == index and rels_data['Formed/Produced'][i] == 1:
                    products.append(rels_data['Reagent'][i])
            reagents = ''
            if len(precursors) == 0:
                reagents += f'{precursors}[0]'
            else:
                for i in range(len(precursors)-1):
                    reagents += f'{precursors[i]} + '
                reagents += f'{precursors[-1]}'
            produced = ''
            if len(products) == 0:
                produced += f'{products}[0]'
            else:
                for i in range(len(products)-1):
                    produced += f'{products[i]} + '
                produced += f'{products[-1]}'
            pathway.append([index, rule, reagents, '->', produced])
            pathway.append(precursor)
    return(pathway)

In [10]:
def pathways(matches, rels):
    matches_data = pd.read_csv(matches, sep='\t')
    rels_data = pd.read_csv(rels, sep='\t')
    pathways = []
    for i in range(1): #len(matches_data['Smiles'])
        pathways.append((pathway_finder(matches_data['Smiles'][i], rels_data)))
        print(i)
    #G0 = ['N', 'C=O', 'C(CO)=O', 'O']
    G0 = ['C=O', 'C(CO)=O']
    counter = 0
    for i in range(len(pathways)): 
        if pathways[i][-1] not in G0:
            counter +=1
    print(f'Errors = {counter}')
    print(f'No. matches = {len(pathways)}')
    #matches_data['Pathways'] = pathways
    return(pathways)

In [25]:
%%time
a = pathways('./MatchedDataNoStereo/PyruvicAcid/NoStereoPyruvicAcid_CHNO.tsv', './PyruvicAcidData/PyruvicAcid_rels.tsv')
#df.to_csv(r'PyruvicAcidPathways.tsv', header=None, index=None, sep='\t', mode='a') 

0
Errors = 0
No. matches = 1
CPU times: user 21.7 s, sys: 78.6 ms, total: 21.8 s
Wall time: 21.9 s


In [26]:
a

[['C(C(CO)O)(O)=O',
  ['5_0', 'Hydration of C(=O)C', 'C(C(C)=O)(O)=O + O', '->', 'C(C(CO)O)(O)=O'],
  'C(C(C)=O)(O)=O',
  ['23_0',
   'Retro Aldol',
   'C(C(C)(CC(C(O)=O)=O)O)(O)=O',
   '->',
   'C(C(C)=O)(O)=O + C(C(C)=O)(O)=O'],
  'C(C(C)(CC(C(O)=O)=O)O)(O)=O',
  ['150_0',
   'Benzilic Acid Rearrangement',
   'O + C(C(O)=O)(CC(C(C)=O)=O)=O',
   '->',
   'C(C(C)(CC(C(O)=O)=O)O)(O)=O'],
  'O',
  ['153_0',
   'Benzilic Acid Rearrangement (inverse)',
   'C(C(C)(CC=O)O)(O)=O',
   '->',
   'O + C(CC(C(C)=O)=O)=O'],
  'C(C(C)(CC=O)O)(O)=O',
  ['973_0',
   'Retro Aldol',
   'C(C(CO)C(C(O)=O)(C)O)=O',
   '->',
   'C(C(C)(CC=O)O)(O)=O + C=O'],
  'C(C(CO)C(C(O)=O)(C)O)=O',
  ['8845_0',
   'Retro Aldol',
   'C(C(CO)(CO)C(C(O)=O)(C)O)=O',
   '->',
   'C=O + C(C(CO)C(C(O)=O)(C)O)=O'],
  'C(C(CO)(CO)C(C(O)=O)(C)O)=O',
  ['1509_0',
   'Knoevenagel C',
   'C=O + C(C(CO)C(C(O)=O)(C)O)=O',
   '->',
   'C(C(CO)(CO)C(C(O)=O)(C)O)=O'],
  'C=O']]

In [15]:
FormoseAmm = pd.read_csv('./FormoseAmmData/FormoseAmm_rels.tsv', sep='\t')

In [23]:
indexes = []
for i in range(len(FormoseAmm['Reagent'])):
    if FormoseAmm['Reagent'][i] == 'C(C(C(C(C(CO)O)O)O)N)=O' and FormoseAmm['Formed/Produced'][i] == 1:
        indexes.append(FormoseAmm['Index'][i])

In [24]:
indexes

['21_0',
 '21_1',
 '17530_0',
 '29108_0',
 '34477_0',
 '34478_0',
 '34479_0',
 '34480_0',
 '34481_0',
 '34482_0',
 '34483_0',
 '34484_0',
 '34485_0',
 '34486_0',
 '34487_0',
 '34488_0',
 '34489_0',
 '34490_0',
 '34491_0',
 '34492_0',
 '34493_0',
 '34494_0',
 '34495_0',
 '34496_0',
 '34497_0',
 '34498_0',
 '34499_0',
 '34500_0',
 '34501_0',
 '34502_0',
 '34503_0',
 '34504_0',
 '34505_0',
 '34506_0',
 '41743_0',
 '41743_1',
 '41902_0',
 '41904_0',
 '41906_0',
 '41912_0',
 '42032_0',
 '42033_0',
 '49754_0',
 '51429_0',
 '51440_0',
 '51444_0',
 '53311_0',
 '53584_0',
 '53846_0',
 '54394_0',
 '54513_0',
 '54632_0',
 '73436_0',
 '73438_0',
 '73440_0',
 '73442_0',
 '73444_0',
 '73446_0',
 '73448_0',
 '73450_0',
 '73452_0',
 '73454_0',
 '73456_0',
 '73458_0',
 '73460_0',
 '73462_0',
 '73464_0',
 '73466_0',
 '73468_0',
 '73470_0',
 '73472_0',
 '73474_0',
 '73476_0',
 '73478_0',
 '73480_0',
 '73482_0',
 '73484_0',
 '73486_0',
 '73488_0',
 '73490_0',
 '73492_0',
 '73494_0',
 '97721_0']